1. 调用必要模块，定义必要常数

In [1]:
from PIL import Image
import numpy as np
import time
DIMENSION = 256
PIC_NUM = 100

2. 获取所有图片的RGB矩阵

In [2]:
start_time = time.time()
RED_init = []
GREEN_init = []
BLUE_init = []
for i in range(PIC_NUM):
    if i < 10:
        i = str(i)
        img_fn = "Images/agricultural/agricultural0" + i + ".tif"
    else:
        i = str(i)
        img_fn = "Images/agricultural/agricultural" + i + ".tif"

    img = Image.open(img_fn)
    rgb_img = img.convert("RGB")
    im = np.array(rgb_img)
    Red = im[:,:,0]
    Green = im[:,:,1]
    Blue = im[:,:,2]
    RED_init.append(Red)
    GREEN_init.append(Green)
    BLUE_init.append(Blue)

# 转化为numpy列表
RED_init = np.array(RED_init) # 存储每张图片的R矩阵
GREEN_init = np.array(GREEN_init) # 存储每张图片的G矩阵
BLUE_init = np.array(BLUE_init) # 存储每张图片的B矩阵

3. 编写一个可以让样本去中心化，且计算样本协方差矩阵的函数

In [3]:
def Cov_calculation(Red_cal, Green_cal, Blue_cal):
    average_red = np.mean(Red_cal, axis=0)
    average_green = np.mean(Green_cal, axis=0)
    average_blue = np.mean(Blue_cal, axis=0)
    Red_Sample = Red_cal - average_red
    Green_Sample = Green_cal - average_green
    Blue_Sample = Blue_cal - average_blue
    Cov_Red = np.matmul(Red_Sample, Red_Sample.T) / (DIMENSION - 1)
    Cov_Green = np.matmul(Green_Sample, Green_Sample.T) / (DIMENSION - 1)
    Cov_Blue = np.matmul(Blue_Sample, Blue_Sample.T) / (DIMENSION - 1)
    return average_red, average_green, average_blue, Red_Sample, Green_Sample, Blue_Sample, Cov_Red, Cov_Green, Cov_Blue

4. 编写一个可以求矩阵特征值与特征向量的函数（这里使用了论文中的QR分解方法）

In [4]:
def eigenvalue_func(cov):
    A_pre = np.zeros((DIMENSION, DIMENSION), dtype=float)
    A_post = cov
    S = np.eye(DIMENSION, dtype=float)
    while np.linalg.norm(np.diag(A_post) - np.diag(A_pre)) >= 0.00001:
        A_pre = A_post
        Q, R = np.linalg.qr(A_post)
        S = np.matmul(S, Q)
        A_post = np.matmul(R, Q)
    return np.diag(A_post), S

5. 选取$k$个主成分

In [5]:
def num_com(eigen):
    k = 0 # 主成分的个数
    k_eigen = 0
    eigen_sum = np.sum(eigen)
    for i in eigen:
        k_eigen = k_eigen + i
        k = k + 1
        if k_eigen / eigen_sum >= 0.9:
            break
    return k

6. 对图片进行压缩，计算运行时间与重构误差

In [6]:
Error = 0
for i in range(PIC_NUM):
    Red_cal = RED_init[i]
    Green_cal = GREEN_init[i]
    Blue_cal = BLUE_init[i]
    average_red, average_green, average_blue, Red_Sample, Green_Sample, Blue_Sample, Cov_Red, Cov_Green, Cov_Blue = Cov_calculation(Red_cal, Green_cal, Blue_cal)
    Red_eigenvalue, Red_eigenvector = eigenvalue_func(Cov_Red)
    Green_eigenvalue, Green_eigenvector = eigenvalue_func(Cov_Green)
    Blue_eigenvalue, Blue_eigenvector = eigenvalue_func(Cov_Blue)
    k_Red = num_com(Red_eigenvalue)
    k_Green = num_com(Green_eigenvalue)
    k_Blue = num_com(Blue_eigenvalue)
    Slice_Red = slice(0, k_Red, 1)
    Slice_Green = slice(0, k_Green, 1)
    Slice_Blue = slice(0, k_Blue, 1)
    S_Red = Red_eigenvector.T[Slice_Red]
    S_Green = Green_eigenvector.T[Slice_Green]
    S_Blue = Blue_eigenvector.T[Slice_Blue]
    W_Red = S_Red.T
    W_Green = S_Green.T
    W_Blue = S_Blue.T
    Red_result = np.matmul(W_Red.T, Red_Sample)
    Red_result = np.matmul(W_Red, Red_result)
    Red_result = Red_result + average_red
    Green_result = np.matmul(W_Green.T, Green_Sample)
    Green_result = np.matmul(W_Green, Green_result)
    Green_result = Green_result + average_green
    Blue_result = np.matmul(W_Blue.T, Blue_Sample)
    Blue_result = np.matmul(W_Blue, Blue_result)
    Blue_result = Blue_result + average_blue
    im1 = np.hstack((Red_result, Green_result, Blue_result))

    im3_channels = np.hsplit(im1, 3)
    im4 = np.zeros_like(im)
    for j in range(3):
        im4[:,:,j] = im3_channels[j]
    
    if i < 10:
        k = str(i)
        Image.fromarray(im4).save("Result/agricultural/agricultural0" + k + ".tif")
    else:
        k = str(i)
        Image.fromarray(im4).save("Result/agricultural/agricultural" + k + ".tif")
    
    Red_Error = Red_result - Red_cal
    Green_Error = Green_result - Green_cal
    Blue_Error = Blue_result - Blue_cal

    Error = Error + np.linalg.norm(Red_Error, ord="fro") + np.linalg.norm(Green_Error, ord="fro") + np.linalg.norm(Blue_Error, ord="fro")
end_time = time.time()
print("Running time:" , end_time - start_time, "seconds")
print(Error)

Running time: 6680.422995328903 seconds
447548.7956132614


7. 计算agricultural00.tif图像的重构误差

In [7]:
start_time = time.time()
Red_cal = RED_init[0]
Green_cal = GREEN_init[0]
Blue_cal = BLUE_init[0]
average_red, average_green, average_blue, Red_Sample, Green_Sample, Blue_Sample, Cov_Red, Cov_Green, Cov_Blue = Cov_calculation(Red_cal, Green_cal, Blue_cal)
Red_eigenvalue, Red_eigenvector = eigenvalue_func(Cov_Red)
Green_eigenvalue, Green_eigenvector = eigenvalue_func(Cov_Green)
Blue_eigenvalue, Blue_eigenvector = eigenvalue_func(Cov_Blue)
k_Red = num_com(Red_eigenvalue)
k_Green = num_com(Green_eigenvalue)
k_Blue = num_com(Blue_eigenvalue)
Slice_Red = slice(0, k_Red, 1)
Slice_Green = slice(0, k_Green, 1)
Slice_Blue = slice(0, k_Blue, 1)
S_Red = Red_eigenvector.T[Slice_Red]
S_Green = Green_eigenvector.T[Slice_Green]
S_Blue = Blue_eigenvector.T[Slice_Blue]
W_Red = S_Red.T
W_Green = S_Green.T
W_Blue = S_Blue.T
Red_result = np.matmul(W_Red.T, Red_Sample)
Red_result = np.matmul(W_Red, Red_result)
Red_result = Red_result + average_red
Green_result = np.matmul(W_Green.T, Green_Sample)
Green_result = np.matmul(W_Green, Green_result)
Green_result = Green_result + average_green
Blue_result = np.matmul(W_Blue.T, Blue_Sample)
Blue_result = np.matmul(W_Blue, Blue_result)
Blue_result = Blue_result + average_blue

Red_Error = Red_result - Red_cal
Green_Error = Green_result - Green_cal
Blue_Error = Blue_result - Blue_cal

Error = np.linalg.norm(Red_Error, ord="fro") + np.linalg.norm(Green_Error, ord="fro") + np.linalg.norm(Blue_Error, ord="fro")
print(Error)
end_time = time.time()
print("Running time:",end_time - start_time,"seconds")

5338.124808206312
Running time: 113.82537364959717 seconds
